In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.tz import tzlocal

# caiman
import caiman as cm
from caiman.paths import caiman_datadir
from caiman.utils.utils import download_demo
from caiman.source_extraction.cnmf import params as params
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf

In [ ]:
caiman_datadir()

In [ ]:
fnames = [os.path.join(
    caiman_datadir(), 
    'example_movies',
    'Sue_2x_3000_40_-46.nwb'
)]
# estimates save path can be same or different from raw data path

save_path = os.path.join(
    caiman_datadir(), 
    'example_movies',
    'Sue_2x_3000_40_-46_CNMF_estimates.nwb'
)
# filename to be created or processed

# dataset dependent parameters
fr = 15.  # imaging rate in frames per second
decay_time = 0.4  # length of a typical transient in seconds

In [ ]:
# NWBがない場合保存する
if not os.path.exists(fnames[0]):
    
    # tifロードする
    fnames_orig = 'Sue_2x_3000_40_-46.tif'
    if fnames_orig in ['Sue_2x_3000_40_-46.tif', 'demoMovie.tif']:
        fnames_orig = [download_demo(fnames_orig)]
    orig_movie = cm.load(fnames_orig, fr=fr)

    # save file in NWB format with various additional info
    orig_movie.save(
        fnames[0], 
        sess_desc='test', 
        identifier='demo 1',
        imaging_plane_description='single plane',
        emission_lambda=520.0, indicator='GCAMP6f',
        location='parietal cortex',
        experimenter='Sue Ann Koay', lab_name='Tank Lab',
        institution='Princeton U',
        experiment_description='Experiment Description',
        session_id='Session 1',
        var_name_hdf5='TwoPhotonSeries')

In [ ]:
# motion_correction parameter
dxy = (2., 2.)  # spatial resolution in x and y in (um per pixel)
# note the lower than usual spatial resolution here
max_shift_um = (12., 12.)  # maximum shift in um
patch_motion_um = (100., 100.)  # patch size for non-rigid correction in um
pw_rigid = True       # flag to select rigid vs pw_rigid motion correction
# maximum allowed rigid shift in pixels
max_shifts = [int(a/b) for a, b in zip(max_shift_um, dxy)]
# start a new patch for pw-rigid motion correction every x pixels
strides = tuple([int(a/b) for a, b in zip(patch_motion_um, dxy)])
# overlap between patches (size of patch in pixels: strides+overlaps)
overlaps = (24, 24)
# maximum deviation allowed for patch with respect to rigid shifts
max_deviation_rigid = 3

mc_dict = {
    'fnames': fnames,
    'fr': fr,
    'decay_time': decay_time,
    'dxy': dxy,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': 'copy',
    #'var_name_hdf5': 'acquisition/TwoPhotonSeries'
    'var_name_hdf5': 'TwoPhotonSeries'
}

opts = params.CNMFParams(params_dict=mc_dict)

### motion correction

In [ ]:
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [ ]:
mc = MotionCorrect(
    fnames, 
    dview=dview, 
    var_name_hdf5=opts.data['var_name_hdf5'], 
    **opts.get_group('motion')
)

In [ ]:
mc.motion_correct(save_movie=True)

In [ ]:
# %% MEMORY MAPPING
border_to_0 = 0 if mc.border_nan == 'copy' else mc.border_to_0
# you can include the boundaries of the FOV if you used the 'copy' option
# during motion correction, although be careful about the components near
# the boundaries

# memory map the file in order 'C'
fname_new = cm.save_memmap(
    mc.mmap_file, base_name='memmap_', order='C',
    border_to_0=border_to_0)  # exclude borders

In [ ]:
# now load the file
Yr, dims, T = cm.load_memmap(fname_new)
images = np.reshape(Yr.T, [T] + list(dims), order='F')

### CNMF

In [ ]:
# parameter
p = 1                    # order of the autoregressive system
gnb = 2                  # number of global background components
merge_thr = 0.85         # merging threshold, max correlation allowed
rf = 15
# half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
stride_cnmf = 6          # amount of overlap between the patches in pixels
K = 4                    # number of components per patch
gSig = [4, 4]            # expected half size of neurons in pixels
# initialization method (if analyzing dendritic data using 'sparse_nmf')
method_init = 'greedy_roi'
ssub = 2                     # spatial subsampling during initialization
tsub = 2                     # temporal subsampling during intialization

# parameters for component evaluation
opts_dict = {
    'fnames': fnames,
     'fr': fr,
     'nb': gnb,
     'rf': rf,
     'K': K,
     'gSig': gSig,
     'stride': stride_cnmf,
     'method_init': method_init,
     'rolling_sum': True,
     'merge_thr': merge_thr,
     'n_processes': n_processes,
     'only_init': True,
     'ssub': ssub,
     'tsub': tsub
}

opts.change_params(params_dict=opts_dict);
# %% RUN CNMF ON PATCHES
# First extract spatial and temporal components on patches and combine them
# for this step deconvolution is turned off (p=0)
opts.change_params({'p': 0})

In [ ]:
cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
cnm = cnm.fit(images)

In [ ]:
# %% plot contours of found components
Cn = cm.local_correlations(images, swap_dim=False)
Cn[np.isnan(Cn)] = 0
cnm.estimates.plot_contours(img=Cn)
plt.title('Contour plots of found components')

#%% save results in a separate file (just for demonstration purposes)
cnm.estimates.Cn = Cn
# cnm.save(fname_new[:-4]+'hdf5')
#cm.movie(Cn).save(fname_new[:-5]+'_Cn.tif'

In [ ]:
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)

In [ ]:
cnm.estimates.save_NWB(
    save_path, 
    imaging_rate=fr, 
    session_start_time=datetime.now(tzlocal()), 
    raw_data_file=fnames[0]
)

In [ ]:
fr

### NWB保存

In [ ]:
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.base import Images
from pynwb.image import GrayscaleImage
from pynwb.device import Device
from pynwb.ophys import OpticalChannel, ImageSeries, ImageSegmentation, Fluorescence

In [ ]:
# NWBFile作成
identifier = 'CaImAn'
session_start_time=datetime.now(tzlocal())
exp_desc=None

nwbfile = NWBFile(
    'CaImAn Results', 
    identifier, 
    session_start_time, 
    experiment_description=exp_desc
)

In [ ]:
# deviceを追加
device = Device('imaging_device')
nwbfile.add_device(device)

In [ ]:
emission_lambda=520.0

imaging_plane_description='some imaging plane description'

optical_channel = OpticalChannel(
    'OpticalChannel',
    'main optical channel',
    emission_lambda=emission_lambda
)

In [ ]:
excitation_lambda=488.0
imaging_rate=30.
indicator='OGB-1'
location='brain'

nwbfile.create_imaging_plane(
    name='ImagingPlane',
    optical_channel=optical_channel,
    description=imaging_plane_description,
    device=device,
    excitation_lambda=excitation_lambda,
    imaging_rate=imaging_rate,
    indicator=indicator,
    location=location
)

In [ ]:
nwbfile.add_acquisition(
    ImageSeries(
        name='TwoPhotonSeries',
        external_file=[fnames[0]],
        format='external',
        rate=imaging_rate,
        starting_frame=[0])
)

### ophys

In [ ]:
mod = nwbfile.create_processing_module('ophys', 'contains caiman estimates for the main imaging plane')

In [ ]:
img_seg = ImageSegmentation()
mod.add(img_seg)

In [ ]:
mod.add_data_interface(fl)

In [ ]:
imaging_plane = list(nwbfile.imaging_planes.values())[0]
imaging_plane

In [ ]:
image_series = list(nwbfile.acquisition.values())[0]
image_series

In [ ]:
ps = img_seg.create_plane_segmentation(
    name='PlaneSegmentation',
    description='CNMF_ROIs',
    imaging_plane=imaging_plane,
    reference_images=image_series
)

In [ ]:
ps.add_column('r', 'description of r values')
ps.add_column('snr', 'signal to noise ratio')
ps.add_column('accepted', 'in accepted list')
ps.add_column('rejected', 'in rejected list')

In [ ]:
if cnm.estimates.cnn_preds is not None:
    ps.add_column('cnn', 'description of CNN')
if cnm.estimates.idx_components is not None:
    ps.add_column('keep', 'in idx_components')

### Add ROIsを追加

In [ ]:
for i in range(cnm.estimates.A.shape[-1]):
    add_roi_kwargs = dict(
        image_mask=cnm.estimates.A.T[i].T.toarray().reshape(cnm.estimates.dims), 
        r=cnm.estimates.r_values[i], 
        snr=cnm.estimates.SNR_comp[i], 
        accepted=False, 
        rejected=False
    )

    if hasattr(cnm.estimates, 'accepted_list'):
        add_roi_kwargs.update(accepted=i in cnm.estimates.accepted_list)
    if hasattr(cnm.estimates, 'rejected_list'):
        add_roi_kwargs.update(rejected=i in cnm.estimates.rejected_list)
    if cnm.estimates.cnn_preds is not None:
        add_roi_kwargs.update(cnn=cnm.estimates.cnn_preds[i])
    if cnm.estimates.idx_components is not None:
        add_roi_kwargs.update(keep=i in cnm.estimates.idx_components)

    ps.add_roi(**add_roi_kwargs)

### backgroundsを追加

In [ ]:
# Backgrounds
for bg in cnm.estimates.b.T:
    add_bg_roi_kwargs = dict(
        image_mask=bg.reshape(cnm.estimates.dims),
        r=np.nan,
        snr=np.nan,
        accepted=False,
        rejected=False
    )
    if 'keep' in ps.colnames:
        add_bg_roi_kwargs.update(keep=False)
    if 'cnn' in ps.colnames:
        add_bg_roi_kwargs.update(cnn=np.nan)
    ps.add_roi(**add_bg_roi_kwargs)

In [ ]:
# Add Traces
n_rois = cnm.estimates.A.shape[-1]
n_bg = len(cnm.estimates.f)
rt_region_roi = ps.create_roi_table_region(
    'ROIs', region=list(range(n_rois)))

In [ ]:
rt_region_bg = ps.create_roi_table_region(
    'Background', region=list(range(n_rois, n_rois+n_bg)))

In [ ]:
starting_time = 0
timestamps = np.arange(cnm.estimates.f.shape[1]) / imaging_rate + starting_time

# Neurons
fl = Fluorescence()
fl.create_roi_response_series(
    name='RoiResponseSeries', data=cnm.estimates.C.T, rois=rt_region_roi, unit='lumens', timestamps=timestamps)
# Background
fl.create_roi_response_series(
    name='Background_Fluorescence_Response', data=cnm.estimates.f.T, rois=rt_region_bg, unit='lumens', timestamps=timestamps)

mod.add(TimeSeries(
    name='residuals', description='residuals', data=cnm.estimates.YrA.T, timestamps=timestamps, unit='NA'))

In [ ]:
if hasattr(cnm.estimates, 'Cn'):
    images = Images('summary_images')
    images.add_image(GrayscaleImage(name='local_correlations', data=cnm.estimates.Cn))

In [ ]:
with NWBHDF5IO(save_path, 'w') as io:
    io.write(nwbfile)

In [ ]:
with NWBHDF5IO(save_path, 'r') as io:
    nwbfile = io.read()

In [ ]:
import h5py
def PrintOnlyDataset(name, obj):
    if isinstance(obj, h5py.Dataset):
        print(name)

with h5py.File(save_path, "r") as f:
    f.visititems(PrintOnlyDataset)

### cnm2

In [ ]:
cnm2.estimates.plot_contours(img=Cn, idx=cnm.estimates.idx_components)

In [ ]:
cnm.params.change_params({'p': p})
cnm2 = cnm.refit(images, dview=dview)

In [ ]:
# %% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier

min_SNR = 2  # signal to noise ratio for accepting a component
rval_thr = 0.85  # space correlation threshold for accepting a component
cnn_thr = 0.99  # threshold for CNN based classifier
cnn_lowest = 0.1 # neurons with cnn probability lower than this value are rejected

cnm2.params.set(
    'quality', {
        'decay_time': decay_time, 
        'min_SNR': min_SNR,
        'rval_thr': rval_thr,
        'use_cnn': True,
        'min_cnn_thr': cnn_thr,
        'cnn_lowest': cnn_lowest
    }
);
cnm2.estimates.evaluate_components(images, cnm2.params, dview=dview)
#%%
cnm2.estimates.Cn = Cn
# cnm2.save(fname_new[:-4] + 'hdf5')
# %% PLOT COMPONENTS
cnm2.estimates.plot_contours(img=Cn, idx=cnm2.estimates.idx_components)

In [ ]:
#%% Extract DF/F values
cnm2.estimates.detrend_df_f(quantileMin=8, frames_window=250)

In [ ]:
#%% Show final traces
cnm2.estimates.view_components(img=Cn)

In [ ]:
cnm2.estimates.save_NWB(
    save_path, 
    imaging_rate=fr, 
    session_start_time=datetime.now(tzlocal()), 
    raw_data_file=fnames[0]
)